#Optimization of Sparse Deep Neural Network using GPU



##**Project Final Phase**

### GROUP MEMBERS:


1.   Amruta Gokhale (12040140)
2.   Ankita Kumari (12040220)
3.   Nidhi Sinchana SR (12040970)


In [ ]:
! unzip "/content/drive/MyDrive/Parallelization Project/neuron1024.zip"

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ctemy5nc
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ctemy5nc
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4307 sha256=8a2a1db4e3fa2d406bffa64dfc071941be4ca7cff487d106f6b420f7bb3d4e82
  Stored in directory: /tmp/pip-ephem-wheel-cache-5j3ybap_/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>  
#include <cuda.h>

__global__ void neuralLayers(float *layerMatrix, float *inputMatrix, int *inputEmptyRows, int *inputNonZeroValues, float *nextLayerInput, int *swapEmptyRows){
    
    __shared__ float inputShared[32][32];
    __shared__ float layerShared[32][32];

    int column = blockIdx.y*32 + threadIdx.y;
    int row = blockIdx.x*32 + threadIdx.x;

    float calculatedValue = 0;
    
if (inputEmptyRows[row]!=1){
     for (int m=0; m<1024/32; m++){
        int sharedRow = threadIdx.x+(blockIdx.x*32);
        int sharedColumn = (32*m)+threadIdx.y;
        int layerRow = (32*m)+threadIdx.x;
        int layerColumn = threadIdx.y+(blockIdx.y*32);
        inputShared[threadIdx.x][threadIdx.y]= inputMatrix[(1024*sharedRow)+sharedColumn];
        layerShared[threadIdx.x][threadIdx.y]= layerMatrix[(1024*layerColumn)+layerRow];
        
        __syncthreads();

        for (int k=0;k<32;k++){
            calculatedValue+=inputShared[threadIdx.x][k]*(0.0625*layerShared[k][threadIdx.y]);
        }

    
     }
     if (calculatedValue!=0){
            calculatedValue=calculatedValue-0.3;
        } 

        if (calculatedValue<0){
            calculatedValue=0;
        } else if(calculatedValue>32){
            calculatedValue=32;
        }
}
     if (calculatedValue!=0){
         if (swapEmptyRows[row]==1){
             swapEmptyRows[row]=0;
         } 
         atomicAdd(&inputNonZeroValues[row], 1);
     }
     nextLayerInput[row*1024+column] = calculatedValue;
   
  }


int main(){
    
    float *wMatrix1, *wMatrix2, *gpu_wMatrix;
    
    FILE *y_file = fopen("/content/drive/MyDrive/Parallelization Project/sparse-images-1024.tsv", "r");

    char buffer[1024];
    int *emptyRows, firstvertex, *nonZeroCount, secondvertex, *gpu_emptyRows, *gpu_nonZeroCount, *gpu_swapEmptyRows, *swapEmptyRows;
    float *gpu_y0, *gpu_nexty0;

    emptyRows = (int *)malloc(sizeof(int)*(60000));
    swapEmptyRows = (int *)malloc(sizeof(int)*60000);
    nonZeroCount = (int *)malloc(sizeof(int)*60000);

    float* y0 = (float *)malloc(sizeof(float)*60000*1024);
    float* y1 = (float *)malloc(sizeof(float)*60000*1024);

    for (int p=0;p<1024*60000;p++){
            y0[p]=0;
    }
    
    for (int i=0; i<60000; i++){
      emptyRows[i]=1;
      swapEmptyRows[i]=1;
      nonZeroCount[i]=0;
    }

    while (fgets(buffer, sizeof(buffer),y_file)!=NULL){
    
    firstvertex=atoi(strtok(buffer, "\t"));
    secondvertex=atoi(strtok(NULL, "\t"));

    if (emptyRows[firstvertex-1]==1){
        emptyRows[firstvertex-1]=0;
    }
    y0[(secondvertex-1)+((firstvertex-1)*1024)]=1;
    
    
	}
	fclose(y_file);

    cudaMalloc(&gpu_y0, 1024*60000*sizeof(float));
    cudaMalloc(&gpu_emptyRows, 60000*sizeof(int));
    cudaMalloc(&gpu_swapEmptyRows, 60000*sizeof(int));
    cudaMalloc(&gpu_nonZeroCount, 60000*sizeof(int));
    cudaMalloc(&gpu_nexty0, 1024*60000*sizeof(float));

    cudaMemcpy(gpu_y0, y0, 1024*60000*sizeof(float), cudaMemcpyHostToDevice) ;
    cudaMemcpy(gpu_emptyRows, emptyRows, 60000*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_swapEmptyRows, swapEmptyRows, 60000*sizeof(int), cudaMemcpyHostToDevice);

    int fileNumber=0;
    cudaMalloc(&gpu_wMatrix, 1024*1024*sizeof(float));

    while (fileNumber<1919){
        
        fileNumber++;

        char *fileToOpen="/content/neuron1024/n1024-l";
        char fileNumberString[50];

        if (fileNumber==1){
            snprintf(fileNumberString, 100, "/content/neuron1024/n1024-l%d", fileNumber);
            strcat(fileNumberString, ".tsv");

            char chunk[2048];
            int maxvertex = 1024;
            int firstnode, secondnode;

            cudaMallocHost(&wMatrix1, sizeof(float)*1024*1024);

            FILE *wFile1 = fopen(fileNumberString, "r");

            while (fgets(chunk, sizeof(chunk),wFile1)!=NULL){
                
                firstnode=atoi(strtok(chunk, "\t"));
                secondnode = atoi(strtok(NULL, "\t"));

                wMatrix1[(secondnode-1)+((firstnode-1)*1024)]=1;
                
            }
            fclose(wFile1);    
        

            cudaMemcpy(gpu_wMatrix, wMatrix1, (sizeof(float)*1024*1024), cudaMemcpyHostToDevice);

            strcpy(fileNumberString, "");
            snprintf(fileNumberString, 100, "/content/neuron1024/n1024-l%d", fileNumber+1);
            strcat(fileNumberString, ".tsv");


            cudaMallocHost(&wMatrix2, sizeof(float)*1024*1024);

            FILE *wFile2 = fopen(fileNumberString, "r");

            while (fgets(chunk, sizeof(chunk),wFile2)!=NULL){
                
                firstnode=atoi(strtok(chunk, "\t"));
                secondnode = atoi(strtok(NULL, "\t"));

                wMatrix2[(secondnode-1)+((firstnode-1)*1024)]=1;
                
            }
            fclose(wFile2);

        } else{
            snprintf(fileNumberString, 100, "/content/neuron1024/n1024-l%d", fileNumber+1);
            strcat(fileNumberString, ".tsv");

            if (fileNumber%2==0){
                cudaFreeHost(wMatrix1);

                cudaMemcpy(gpu_wMatrix, wMatrix2, (sizeof(float)*1024)*1024, cudaMemcpyHostToDevice);

                FILE *w_file1 = fopen(fileNumberString, "r");

                char chunk[2048];
                int maxvertex = 1024;
                int firstnode, secondnode;

                cudaMallocHost(&wMatrix1, sizeof(float)*1024*1024);

                FILE *wFile1 = fopen(fileNumberString, "r");

                while (fgets(chunk, sizeof(chunk),wFile1)!=NULL){
                    
                    firstnode=atoi(strtok(chunk, "\t"));
                    secondnode = atoi(strtok(NULL, "\t"));

                    wMatrix1[(secondnode-1)+((firstnode-1)*1024)]=1;
                    
                }
                fclose(wFile1);
                

            } else{
                cudaFreeHost(wMatrix2);


                cudaMemcpy(gpu_wMatrix, wMatrix1, (sizeof(float)*1024*1024), cudaMemcpyHostToDevice);

                FILE *w_file2 = fopen(fileNumberString, "r");

                char chunk[2048];
                int maxvertex = 1024;
                int firstnode, secondnode;


                cudaMallocHost(&wMatrix2, sizeof(float)*1024*1024);

                FILE *wFile2 = fopen(fileNumberString, "r");

                while (fgets(chunk, sizeof(chunk),wFile2)!=NULL){
                    
                    firstnode=atoi(strtok(chunk, "\t"));
                    secondnode = atoi(strtok(NULL, "\t"));

                    wMatrix2[(secondnode-1)+((firstnode-1)*1024)]=1;
                    
                }
                fclose(wFile2);
            }
        }

        dim3 block(32, 32);
        dim3 grid(60000/32, 1024/32);

        neuralLayers<<<grid, block>>>(gpu_wMatrix, gpu_y0, gpu_emptyRows, gpu_nonZeroCount, gpu_nexty0, gpu_swapEmptyRows);
        cudaMemcpy(gpu_y0, gpu_nexty0, 1024*60000*sizeof(float), cudaMemcpyDeviceToDevice);
        cudaMemcpy(gpu_emptyRows, gpu_swapEmptyRows, 60000*sizeof(int), cudaMemcpyDeviceToDevice);
        cudaMemcpy(gpu_swapEmptyRows, swapEmptyRows, 60000*sizeof(int), cudaMemcpyHostToDevice);
        cudaMemcpy(y1, gpu_nexty0, sizeof(float)*1024*60000, cudaMemcpyDeviceToHost);

        if (fileNumber+1==1920){
            cudaMemcpy(nonZeroCount, gpu_nonZeroCount, 60000*sizeof(int), cudaMemcpyDeviceToHost);
        }
     
    }


for (int x=0; x<50; x++){
  printf("%f\n", y1[x]);
}    

  return 0;
}


Google drive link:

https://drive.google.com/drive/folders/1QIn80agnIeqojC4f4JYDxT6AQosx0qE4?usp=share_link